In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, TemplateLibrary

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("my-building")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
TemplateLibrary.load(ontology_graph="../libraries/brick/Brick-subset.ttl")

TemplateLibrary(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3-subset/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f9e1df67970>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = TemplateLibrary.load(directory="../libraries/guideline36/")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'dat', 'occ', 'ztemp', 'co2', 'dmp', 'zone'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_5d11e95d> ;
    brick:hasPart <urn:my-building/dmp_497952a5> ;
    brick:hasPoint <urn:my-building/co2_993a40f5>,
        <urn:my-building/dat_1d9fa9d8>,
        <urn:my-building/occ_75753741>,
        <urn:my-building/ztemp_0578b922> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_9c4bc2cd> ;
    brick:hasPart <urn:my-building/dmp_8678f054> ;
    brick:hasPoint <urn:my-building/co2_72b8e71f>,
        <urn:my-building/dat_4e64e795>,
        <urn:my-building/occ_0be925f7>,
        <urn:my-building/ztemp_2925534d> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_698e8806> ;
    brick:hasPart <urn:my-building/dmp_e80ab839> ;
    brick:hasPoint <urn:my-building/co2_15d88954>,
        <urn:my-building/dat_e548916d>,
        <urn:my-building/occ_00c6fc8c>,
        <urn:my-building/ztemp_fcf42f95> .



In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_9e2560e3> a brick:VAV ;
    brick:feeds <urn:my-building/zone_21bc2c67> ;
    brick:hasPart <urn:my-building/dmp_fed080dd> ;
    brick:hasPoint <urn:my-building/co2_2f7d00ae>,
        <urn:my-building/dat_edb62a7d>,
        <urn:my-building/occ_a053a753>,
        <urn:my-building/ztemp_35ca619e> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name-2562f9a3-inlined_0bae85f6> a brick:HVAC_Zone .

<urn:my-building/name-46984e4d-inlined_940fbcde> a brick:Damper_Position_Command .

<urn:my-building/name-74ba28f0-inlined_81c19eee> a brick:CO2_Level_Sensor .

<urn:my-building/name-89dc0fa1-inlined_5f8fe29f> a brick:Damper ;
    brick:hasPoint <urn:my-building/dmppos-89dc0fa1-inlined_afd47235> .

<urn:my-building/name-a7fad5f1-inlined_b3db5191> a brick:Discharge_Air_Temperature_Sensor .

<urn:my-building/name-b657cd77-inlined_f9b46a2e> a brick:Zone_Air_Temperature_Sensor .

<urn:my-building/name-f0729879-inlined_ca30cbe8> a brick:Occupancy_Sensor .

<urn:my-building/name_2e4b78c1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_1832e92c> ;
    brick:hasPart <urn:my-building/dmp_5013d966> ;
    brick:hasPoint <urn:my-building/co2_ca9685fb>,
        <urn:my-building/dat_a912e760>,
        <urn:my-building/occ_c148a746>,
        <urn:my-building/ztemp_ab3b9